# Customer Churn Modeling Pipeline

This notebook walks through EDA, preprocessing, model training, and evaluation for predicting customer churn using the provided Excel workbook. Copy this notebook to the same folder as `Customer_Churn_Data_Large (1).xlsx` and run cells sequentially.

In [ ]:

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set(style='whitegrid')
pd.set_option('display.max_columns', 200)

# Update filename if needed
file_path = "Customer_Churn_Data_Large (1).xlsx"
xls = pd.ExcelFile(file_path)
print("Sheets available:", xls.sheet_names)

# Load sheets
demographics = pd.read_excel(xls, sheet_name='Customer_Demographics')
transactions = pd.read_excel(xls, sheet_name='Transaction_History')
service = pd.read_excel(xls, sheet_name='Customer_Service')
online = pd.read_excel(xls, sheet_name='Online_Activity')
churn = pd.read_excel(xls, sheet_name='Churn_Status')

print('\nDemographics shape:', demographics.shape)
print('Transactions shape:', transactions.shape)
print('Service shape:', service.shape)
print('Online activity shape:', online.shape)
print('Churn shape:', churn.shape)

# Quick peek
display(demographics.head())
display(transactions.head())
display(service.head())
display(online.head())
display(churn.head())


## EDA: Data types, missing values and basic stats

In [ ]:

# Data types and missing values
def df_summary(df, name):
    print(f"--- {name} ---")
    print(df.dtypes)
    print('\nMissing values:')
    print(df.isnull().sum())
    print('\nSummary statistics:')
    display(df.describe(include='all').T)
    print('\n\n')

df_summary(demographics, 'Demographics')
df_summary(transactions, 'Transactions')
df_summary(service, 'Service')
df_summary(online, 'Online Activity')
df_summary(churn, 'Churn')


## EDA Visualizations

In [ ]:

# Example visualizations
plt.figure(figsize=(8,5))
sns.histplot(demographics['Age'].dropna(), kde=True, bins=30)
plt.title('Age distribution'); plt.show()

plt.figure(figsize=(6,4))
sns.countplot(x='IncomeLevel', data=demographics, order=demographics['IncomeLevel'].value_counts().index)
plt.title('Income Level distribution'); plt.show()

# Merge a small set to visualize churn by income
demo_churn = demographics.merge(churn, on='CustomerID', how='left')
plt.figure(figsize=(6,4))
sns.barplot(x='IncomeLevel', y='Churn', data=demo_churn, estimator=np.mean)
plt.ylabel('Churn Rate'); plt.title('Churn rate by Income Level'); plt.show()


## Preprocessing & Feature Engineering
- Merge datasets on CustomerID
- Create aggregated transaction features (sum, mean, freq)
- Create service features (counts, last contact etc.)
- Handle missing values, encode categoricals, scale numeric features

In [ ]:

# Merge datasets
# Aggregate transactions per customer (example features)
tx_agg = transactions.groupby('CustomerID').agg({
    'TransactionAmount': ['sum', 'mean', 'count', 'max'],
    'TransactionDate': ['min', 'max']
}).reset_index()
tx_agg.columns = ['CustomerID','tx_sum','tx_mean','tx_count','tx_max','tx_first_date','tx_last_date']

# Service aggregation
svc_agg = service.groupby('CustomerID').agg({
    'ServiceCalls': 'sum',
    'Complaints': 'sum',
    'SatisfactionScore': 'mean'
}).reset_index().rename(columns={'ServiceCalls':'svc_calls','Complaints':'svc_complaints','SatisfactionScore':'svc_sat_mean'})

# Online activity aggregation (if present)
if 'CustomerID' in online.columns:
    on_agg = online.groupby('CustomerID').agg({
        'PageViews': 'sum',
        'Logins': 'sum',
        'LastActiveDate': 'max'
    }).reset_index().rename(columns={'PageViews':'on_pageviews','Logins':'on_logins','LastActiveDate':'on_last_active'})
else:
    on_agg = pd.DataFrame(columns=['CustomerID'])

# Merge all
df = demographics.merge(tx_agg, on='CustomerID', how='left')                 .merge(svc_agg, on='CustomerID', how='left')                 .merge(on_agg, on='CustomerID', how='left')                 .merge(churn, on='CustomerID', how='left')

print('Merged dataset shape:', df.shape)
display(df.head())


In [ ]:

# Handle missing values: simple strategy with justification in markdown
# Numeric: fillna with 0 or median; Categorical: fill with 'Unknown'
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print('Numeric cols sample:', num_cols[:10])
print('Categorical cols sample:', cat_cols[:10])

# Fill numeric NaNs with 0 for monetary/usage aggregates, median for others
for c in ['tx_sum','tx_mean','tx_count','tx_max','svc_calls','svc_complaints','svc_sat_mean','on_pageviews','on_logins']:
    if c in df.columns:
        df[c] = df[c].fillna(0)

for c in cat_cols:
    df[c] = df[c].fillna('Unknown')

# Create recency and tenure features if date columns exist
from datetime import datetime
today = pd.to_datetime('2024-09-30')  # example reference date
if 'tx_last_date' in df.columns and not df['tx_last_date'].isnull().all():
    df['tx_last_date'] = pd.to_datetime(df['tx_last_date'])
    df['tx_recency_days'] = (today - df['tx_last_date']).dt.days
else:
    df['tx_recency_days'] = -1

if 'tx_first_date' in df.columns and not df['tx_first_date'].isnull().all():
    df['tx_first_date'] = pd.to_datetime(df['tx_first_date'])
    df['tx_tenure_days'] = (df['tx_last_date'] - df['tx_first_date']).dt.days.fillna(0)
else:
    df['tx_tenure_days'] = 0

# Example ratio feature
df['tx_amount_per_tx'] = df.apply(lambda row: row['tx_sum']/row['tx_count'] if row['tx_count']>0 else 0, axis=1)

# Ensure target column name 'Churn' exists and is numeric 0/1
df['Churn'] = df['Churn'].fillna(0).astype(int)

display(df.head())


### Encoding categorical variables and scaling numeric features

In [ ]:

# Encoding
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Select features for modeling
features = df.drop(columns=['CustomerID','tx_first_date','tx_last_date'])

# Separate target
y = features['Churn']
X = features.drop(columns=['Churn'])

# Identify numeric and categorical
numeric_feats = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_feats = X.select_dtypes(include=['object','category']).columns.tolist()

print('Numeric features count:', len(numeric_feats))
print('Categorical features count:', len(categorical_feats))

# Simple pipeline using pandas
X_encoded = pd.get_dummies(X, columns=categorical_feats, drop_first=True)

# Scale numeric columns
scaler = StandardScaler()
X_encoded[numeric_feats] = scaler.fit_transform(X_encoded[numeric_feats])

print('Final feature matrix shape:', X_encoded.shape)


## Model training: Random Forest with cross-validation and GridSearchCV

In [ ]:

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)
print('Train/test sizes:', X_train.shape, X_test.shape)

# Baseline Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
baseline_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')
print('Baseline CV ROC-AUC:', baseline_scores.mean())

# Hyperparameter search (small grid for speed)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 3, 5]
}
grid = GridSearchCV(rf, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
grid.fit(X_train, y_train)
print('Best params:', grid.best_params_)
print('Best CV ROC-AUC:', grid.best_score_)

best_rf = grid.best_estimator_
# Evaluate on test
y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)[:,1]
print('Test ROC-AUC:', roc_auc_score(y_test, y_proba))
print('\nClassification report:\n', classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print('\nConfusion Matrix:\n', cm)


## Feature importance & insights

In [ ]:

importances = pd.Series(best_rf.feature_importances_, index=X_encoded.columns).sort_values(ascending=False)
display(importances.head(30))

# Plot top 15
plt.figure(figsize=(10,6))
sns.barplot(x=importances.head(15), y=importances.head(15).index)
plt.title('Top 15 Feature Importances'); plt.tight_layout(); plt.show()


## Recommendations & Next steps
- Use model predictions to target high-risk customers with retention offers.
- Monitor feature drift and retrain periodically.
- Consider advanced models (XGBoost, LightGBM) and imbalance techniques (SMOTE, class weights).
- Deploy model as batch scoring for retention campaigns and integrate with CRM.